In [143]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import AzureOpenAI
from pinecone import Pinecone, ServerlessSpec

In [144]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# pc.create_index(
#     name="rate", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
# )

In [145]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. John Smith',
  'subject': 'Introduction to Computer Science',
  'stars': 5,
  'review': "Dr. Smith is an excellent professor who explains complex topics clearly. His lectures are engaging, and he's always willing to help students during office hours."},
 {'professor': 'Dr. Emily Davis',
  'subject': 'Calculus I',
  'stars': 4,
  'review': 'Dr. Davis is a knowledgeable and patient instructor. The course material is challenging, but she makes it manageable. Her exams are tough but fair.'},
 {'professor': 'Dr. Michael Brown',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': "Dr. Brown knows his stuff, but his teaching style can be a bit dry. It's easy to fall behind if you're not paying attention. The labs were helpful, though."},
 {'professor': 'Dr. Linda Green',
  'subject': 'English Literature',
  'stars': 5,
  'review': 'Dr. Green is passionate about literature and it shows in her teaching. Her classes are thought-provoking and she encourages deep analys

In [155]:
processed_data = []
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

for review in data['reviews']:
    response = client.embeddings.create(
        model="text-embedding-ada-002", input=review["review"]
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })

[-0.014571387, 0.021555914, 0.009041437, -0.030988228, 0.0012719526, 0.029937344, -0.006657727, -0.00082660903, -0.018018795, -0.02150465, -0.0181854, 0.009522024, -0.00078976405, 0.0148277, 0.0028178398, -0.02609265, 0.029040249, -0.009765521, 0.010835627, -0.012450398, -0.038113724, -0.0045271264, -0.0071703526, -0.0035627491, -0.01353332, 0.018134136, 0.012194085, -0.009868046, -0.0051679085, -0.009509208, 0.014430415, 0.002700897, -0.03206474, -0.027707422, 0.0028562867, 0.010636984, -0.018467342, 0.015596638, 0.0021049695, -0.008285314, 0.011252135, -0.022273589, 0.009349013, 0.0043445034, -0.016673153, -0.010880481, 0.0059849056, -0.04101006, -0.023939623, 0.005039752, 0.0034890592, 0.015583823, -0.011015046, -0.017057622, 0.0029588118, -0.013802448, -0.0045912047, 0.021414941, -0.0031238133, -0.022940002, -0.008362208, -0.019313175, -0.018172583, 0.022017276, 0.0002685278, 0.012661857, -0.01541722, 0.016365577, 0.01412284, 0.015340326, 0.013430795, -0.0047770315, -0.010765141, 0

In [147]:
processed_data[0]

{'Values': ([-0.014571387,
   0.021555914,
   0.009041437,
   -0.030988228,
   0.0012719526,
   0.029937344,
   -0.006657727,
   -0.00082660903,
   -0.018018795,
   -0.02150465,
   -0.0181854,
   0.009522024,
   -0.00078976405,
   0.0148277,
   0.0028178398,
   -0.02609265,
   0.029040249,
   -0.009765521,
   0.010835627,
   -0.012450398,
   -0.038113724,
   -0.0045271264,
   -0.0071703526,
   -0.0035627491,
   -0.01353332,
   0.018134136,
   0.012194085,
   -0.009868046,
   -0.0051679085,
   -0.009509208,
   0.014430415,
   0.002700897,
   -0.03206474,
   -0.027707422,
   0.0028562867,
   0.010636984,
   -0.018467342,
   0.015596638,
   0.0021049695,
   -0.008285314,
   0.011252135,
   -0.022273589,
   0.009349013,
   0.0043445034,
   -0.016673153,
   -0.010880481,
   0.0059849056,
   -0.04101006,
   -0.023939623,
   0.005039752,
   0.0034890592,
   0.015583823,
   -0.011015046,
   -0.017057622,
   0.0029588118,
   -0.013802448,
   -0.0045912047,
   0.021414941,
   -0.0031238133,
   -

In [156]:
index = pc.Index('rate')
index.upsert(vectors=processed_data, namespace="nsl")

{'upserted_count': 20}

In [157]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'nsl': {'vector_count': 20}},
 'total_vector_count': 20}